<a href="https://colab.research.google.com/github/aaronjoseph/NLP_Boiler_Plate/blob/main/Sentiment_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install mlflow
#Load the libraries
from mlflow import log_metric, log_param, log_artifacts
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import os
from tqdm import tqdm
from xgboost import XGBClassifier
tqdm.pandas()
from sklearn.model_selection import train_test_split
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
sentiment_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',header=None,error_bad_lines=False,engine='python')
sentiment_data = sentiment_data.sample(n=1_00_000,replace=True)

Skipping line 112675: unexpected end of data


In [ ]:
# Data Cleaning Process
def preprocess(text):
  text=text.lower()
  # remove hyperlinks
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
  #remove hashtag sign
  text=re.sub(r"#","",text)   
  #remove mentions
  text = re.sub(r"(?:\@)\w+", '', text)
  #remove non ascii chars
  text=text.encode("ascii",errors="ignore").decode()
  #remove some puncts (except . ! ?)
  text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
  text=re.sub(r'[!]+','!',text)
  text=re.sub(r'[?]+','?',text)
  text=re.sub(r'[.]+','.',text)
  text=re.sub(r"'","",text)
  text=re.sub(r"\(","",text)
  text=re.sub(r"\)","",text)    
  text=" ".join(text.split())
  return text

def simple_stemmer(text):
  ps=nltk.porter.PorterStemmer()
  text= ' '.join([ps.stem(word) for word in text.split()])
  return text

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
  #set stopwords to english
  stop=set(stopwords.words('english'))
  # Tokenization
  tokenizer=ToktokTokenizer()
  #Setting English stopwords
  stopword_list=nltk.corpus.stopwords.words('english')
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
      filtered_tokens = [token for token in tokens if token not in stopword_list]
  else:
      filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)    
  return filtered_text

In [ ]:
#Apply function
sentiment_data['clean']=sentiment_data[5].progress_apply(preprocess)
sentiment_data['stem']=sentiment_data['clean'].progress_apply(simple_stemmer)
sentiment_data['token'] = sentiment_data['stem'].progress_apply(remove_stopwords) 

100%|██████████| 100000/100000 [00:26<00:00, 3800.70it/s]


In [ ]:
# Splitting the data - Into train_test
X = sentiment_data['token']
y = sentiment_data[0]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42) 
# Count Vectorizer for Bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(X_train)
#transformed test reviews
cv_test_reviews=cv.transform(X_test)
# TFIDF
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(X_train)
#transformed test reviews
tv_test_reviews=tv.transform(X_test)

In [ ]:
# Develop XGBoost Model [Classifier/Regressor]
xgb_cv = XGBClassifier(max_depth=6,n_estimators=1000).fit(cv_train_reviews,y_train)
xgb_tfidf = XGBClassifier(max_depth=6,n_estimators=1000).fit(tv_train_reviews,y_train)

# Prediction
prediction_cv = xgb_cv.predict(cv_test_reviews)
prediction_tfidf = xgb_tfidf.predict(tv_test_reviews) 

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, auc, roc_curve, accuracy_score
from sklearn.model_selection import GridSearchCV
Label_Val = 4

In [ ]:
def scoring(y_true,y_pred):
  # Accuracy 
  print('Accuracy Score', accuracy_score(y_true, y_pred))

  # Recall & Precision  
  print('Precision Score',precision_score(y_true, y_pred,pos_label=Label_Val))
  print('Recall Score', recall_score(y_true, y_pred,pos_label=Label_Val))
  
  # F1 Score
  f1_score(y_true, y_pred, pos_label=Label_Val)
  

In [ ]:
print("Printing Metrics for XGboost Count Vectorizer")
scoring(prediction_cv,y_test)
print("="*120,"\nPrinting Metrics for XGboost TFIDF")
scoring(prediction_tfidf,y_test)

Printing Metrics for XGboost Count Vectorizer
Accuracy Score 1.0
Precision Score 0.0
Recall Score 0.0
Printing Metrics for XGboost TFIDF
Accuracy Score 1.0
Precision Score 0.0
Recall Score 0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


Parameters

**Default parameters**

max_depth=3,
learning_rate=0.1, 
n_estimators=100, 
verbosity=1, 
silent=None, 
objective='reg:squarederror', 
booster='gbtree', 
n_jobs=1, 
nthread=None, 
gamma=0, 
min_child_weight=1, 
max_delta_step=0, 
subsample=1, 
colsample_bytree=1, 
colsample_bylevel=1, 
colsample_bynode=1, 
reg_alpha=0, 
reg_lambda=1,
scale_pos_weight=1, 
base_score=0.5, 
random_state=0, 
seed=None,
missing=None, 
importance_type='gain'

**Explanation of relevant parameters for this kernel.**

* booster: Select the type of model to run at each iteration
* gbtree: tree-based models
* gblinear: linear models
* nthread: default to maximum number of threads available if not set
* objective: This defines the loss function to be minimized

**Parameters for controlling speed**

subsample: Denotes the fraction of observations to be randomly samples for each tree

colsample_bytree: Subsample ratio of columns when constructing each tree.

n_estimators: Number of trees to fit.
Important parameters which control overfiting

learning_rate: Makes the model more robust by shrinking the weights on each step

max_depth: The maximum depth of a tree.

min_child_weight: Defines the minimum sum of weights of all observations 
required in a child.

Tuning the hyper-parameters

GridSearchCV params:

estimator: estimator object

param_grid : dict or list of dictionaries

scoring: A single string or a callable to evaluate the predictions on the test 
set. If None, the estimator’s score method is used.

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
n_jobs: Number of jobs to run in parallel. None means. -1 means using all processors.

cv: cross-validation, None, to use the default 3-fold cross validation. Integer, to specify the number of folds in a (Stratified)KFold.


In [ ]:
# Hyper-parameter Tuning using Grid Search to get better score
# Defination for Hyperparameter tuning
#XGBoost hyper-parameter tuning
def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.5, 0.7],
        'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }
    xgb_model = XGBClassifier()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)
    gsearch.fit(X_train,y_train)
    return gsearch.best_params_

In [ ]:
cv_train_reviews

In [ ]:
# Hyperparameter tuning for Count Vectorizer Model
# hyperParameterTuning(cv_train_reviews,y_train)
"""
Parameters
{'colsample_bytree': 0.5,
 'learning_rate': 0.01,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 100,
 'objective': 'reg:squarederror',
 'subsample': 0.5}
"""

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 370 tasks      | elapsed: 31.0min
[Parallel(n_jobs=-1)]: Done 720 tasks      | elapsed: 62.0min
[Parallel(n_jobs=-1)]: Done 1170 tasks      | elapsed: 112.9min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 153.6min finished


{'colsample_bytree': 0.5,
 'learning_rate': 0.01,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 100,
 'objective': 'reg:squarederror',
 'subsample': 0.5}

Best Parameter for CV Data 




In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)

In [ ]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [ ]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [ ]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [ ]:
# LightGBM

In [ ]:
# Few Models 

In [ ]:
# Stacking 

In [ ]:
# MLFLOW Capture

In [ ]:
# Sentiment Models

In [ ]:
!pip install mlflow --quiet
!pip install pyngrok --quiet

import mlflow

with mlflow.start_run(run_name="MLflow on Colab"):
  mlflow.log_metric("m1", 2.0)
  mlflow.log_param("p1", "mlflow-colab")

# run tracking UI in the background
get_ipython().system_raw("mlflow ui --port 5000 &") # run tracking UI in the background


# create remote tunnel using ngrok.com to allow local port access
# borrowed from https://colab.research.google.com/github/alfozan/MLflow-GBRT-demo/blob/master/MLflow-GBRT-demo.ipynb#scrollTo=4h3bKHMYUIG6

from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = ""
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

In [ ]:
# 